In [1]:
try:
    output_filepath
except:
    output_filepath = 's3://workspaces-clarity-mgmt-pro/jaime.oliver/misc/social_capital/'
    input_filepath = '../data/raw/'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from pathlib import Path

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from src.data.migration_network import MigrationNetworkCreation
from src.data.financial_network import IndustryNetworkCreationEORA


In [4]:
year='2015'

def get_networks(year):
    INC = IndustryNetworkCreationEORA(
                year=year, input_filepath='../data/raw/', output_filepath='../data/processed/'
            )
    INC.run()

    MNC = MigrationNetworkCreation(
        year=year, input_filepath='../data/raw/', output_filepath='../data/processed/'
    )
    MNC.run()

    return INC.A, INC.B, MNC.G


A, B, M = get_networks(year = '2015')


In [8]:
G = nx.from_numpy_array(A)

In [14]:
from src.utils.utils_s3 import read_s3_graphml, write_s3_graphml


bucket='workspaces-clarity-mgmt-pro'
s3_path='jaime.oliver/misc/social_capital/data/test/G.graphml'
local_network_path = f'G.graphml'

write_s3_graphml(G, bucket, s3_path)
G_read = read_s3_graphml(bucket, s3_path)